# Topic modeling

##  Extracting key topics of discussion from the topics, displaying top 5 distinct topics, and showing which articles are related to which article.

### Prerequisites – Download nltk stopwords and spacy model

In [4]:
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\arjun\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Import packages

In [3]:
pip install -U gensim

In [2]:
!python -m pip install -U pyLDAvis

Processing c:\users\arjun\appdata\local\pip\cache\wheels\90\61\ec\9dbe9efc3acf9c4e37ba70fbbcc3f3a0ebd121060aa593181a\pyldavis-3.3.1-py2.py3-none-any.whl
  Using cached pandas-1.2.4-cp38-cp38-win_amd64.whl (9.3 MB)
  Using cached funcy-1.16-py2.py3-none-any.whl (32 kB)
Processing c:\users\arjun\appdata\local\pip\cache\wheels\22\0b\40\fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897\sklearn-0.0-py2.py3-none-any.whl


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'C:\\Users\\arjun\\anaconda3\\Lib\\site-packages\\~andas\\_libs\\algos.cp38-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.3
    Uninstalling pandas-1.1.3:
      Successfully uninstalled pandas-1.1.3


In [21]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

### Prepare stop words

In [5]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

### Import Newsgroups Data

In [3]:
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df.target_names.unique())
df.head()

['rec.autos' 'comp.sys.mac.hardware' 'comp.graphics' 'sci.space'
 'talk.politics.guns' 'sci.med' 'comp.sys.ibm.pc.hardware'
 'comp.os.ms-windows.misc' 'rec.motorcycles' 'talk.religion.misc'
 'misc.forsale' 'alt.atheism' 'sci.electronics' 'comp.windows.x'
 'rec.sport.hockey' 'rec.sport.baseball' 'soc.religion.christian'
 'talk.politics.mideast' 'talk.politics.misc' 'sci.crypt']


,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


### Remove emails and newline characters


In [6]:
# Convert to list
data = df.content.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']


#### We need to break down each sentence into a list of words through tokenization, while clearing up all the messy text in the process for LDA to consume.

## Tokenize words and Clean-up text

In [7]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


## Creating Bigram and Trigram Models

In [8]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd_edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


## Remove Stopwords, Make Bigrams and Lemmatize


In [9]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

## Calling the functions in order

In [12]:
import spacy
from spacy.cli.download import download
download(model="en_core_web_sm")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [13]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['where', 's', 'thing', 'car', 'nntp_poste', 'host', 'rac_wam', 'umd', 'organization', 'university', 'park', 'line', 'wonder', 'enlighten', 'car', 'saw', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'bricklin', 'door', 'really', 'small', 'addition', 'separate', 'rest', 'body', 'know', 'tellme', 'model', 'name', 'engine', 'spec', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


## Create the Dictionary and Corpus needed for Topic Modeling

In [14]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 5), (6, 1), (7, 2), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 2), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1)]]


### Gensim creates a unique id for each word in the document. The produced corpus shown above is a mapping of (word_id, word_frequency).
### For example, (0, 1) above implies, word id 0 occurs once in the first document. Likewise, word id 1 occurs twice and so on.
### This is used as the input by the LDA model.
### If you want to see what word a given id corresponds to, pass the id as a key to the dictionary.



In [15]:
id2word[0]


'addition'

In [16]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('addition', 1),
  ('body', 1),
  ('bricklin', 1),
  ('bring', 1),
  ('call', 1),
  ('car', 5),
  ('day', 1),
  ('door', 2),
  ('early', 1),
  ('engine', 1),
  ('enlighten', 1),
  ('funky', 1),
  ('history', 1),
  ('host', 1),
  ('info', 1),
  ('know', 1),
  ('late', 1),
  ('lerxst', 1),
  ('line', 1),
  ('look', 2),
  ('mail', 1),
  ('make', 1),
  ('model', 1),
  ('name', 1),
  ('neighborhood', 1),
  ('nntp_poste', 1),
  ('organization', 1),
  ('park', 1),
  ('production', 1),
  ('rac_wam', 1),
  ('really', 1),
  ('rest', 1),
  ('s', 1),
  ('saw', 1),
  ('separate', 1),
  ('small', 1),
  ('spec', 1),
  ('sport', 1),
  ('tellme', 1),
  ('thank', 1),
  ('thing', 1),
  ('umd', 1),
  ('university', 1),
  ('where', 1),
  ('wonder', 1),
  ('year', 1)]]

## Building the Topic Model

In [17]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

## View the topics in LDA model

In [18]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.587*"ax" + 0.097*"_" + 0.047*"max" + 0.020*"screen" + 0.013*"mouse" + '
  '0.008*"mr" + 0.007*"modem" + 0.006*"el" + 0.005*"cds" + 0.005*"honda"'),
 (1,
  '0.079*"key" + 0.041*"chip" + 0.022*"encryption" + 0.021*"server" + '
  '0.019*"technology" + 0.019*"security" + 0.018*"clipper" + 0.016*"algorithm" '
  '+ 0.016*"controller" + 0.015*"system"'),
 (2,
  '0.033*"self" + 0.028*"normal" + 0.024*"son" + 0.024*"proof" + '
  '0.023*"father" + 0.022*"alone" + 0.021*"muslim" + 0.020*"islam" + '
  '0.020*"iran" + 0.019*"vote"'),
 (3,
  '0.033*"people" + 0.018*"state" + 0.015*"right" + 0.012*"israel" + '
  '0.011*"group" + 0.011*"government" + 0.010*"issue" + 0.008*"death" + '
  '0.008*"kill" + 0.008*"case"'),
 (4,
  '0.030*"trivial" + 0.021*"allen" + 0.018*"click" + 0.016*"ms_window" + '
  '0.004*"rude" + 0.000*"sphere" + 0.000*"steven" + 0.000*"central" + '
  '0.000*"hitachi" + 0.000*"circle"'),
 (5,
  '0.071*"gun" + 0.025*"crime" + 0.023*"police" + 0.022*"weapon" + '
  '0.022*"fire

### How to interpret this?
### Topic 0 is represented as 0.587*"ax" + 0.097*"_" + 0.047*"max" + 0.020*"screen" + 0.013*"mouse" + '
###  '0.008*"mr" + 0.007*"modem" + 0.006*"el" + 0.005*"cds" + 0.005*"honda"'

### It means the top 10 keywords that contribute to this topic are: ‘ax’, ‘max’, ‘screen’.. and so on and the weight of ‘ax’ on topic 0 is 0.587.
### The weights reflect how important a keyword is to that topic.

## Compute Model Perplexity and Coherence Score

In [19]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -14.273084520688098

Coherence Score:  0.5061935618822515


## Visualize the topics-keywords

In [23]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
15     0.380167 -0.052326       1        1  37.351563
14     0.279699  0.249992       2        1  12.685894
9      0.336398 -0.074591       3        1  12.190935
3      0.293085 -0.097040       4        1  10.964722
0     -0.013394 -0.106262       5        1   5.634468
8      0.118515 -0.194212       6        1   4.908728
7      0.018015 -0.000434       7        1   2.556369
1      0.040144  0.258105       8        1   2.434880
13    -0.027191  0.031675       9        1   2.264110
16    -0.023798  0.090097      10        1   1.608811
5     -0.077521 -0.007964      11        1   1.493661
6     -0.079448  0.052072      12        1   1.442047
2     -0.121769 -0.009079      13        1   1.067878
12    -0.102766 -0.060045      14        1   0.968420
18    -0.153508 -0.028789      15        1   0.729622
17    -0.142568 -0.028608      16        1   0.693582
19    -0.148915  0.028711      17        1   0.490831
10    -0.178939 -0.020382      18        1   0.341384
11    -0.195851 -0.016019      19        1   0.120475
4     -0.200355 -0.014900      20        1   0.051620, topic_info=               Term          Freq         Total Category  logprob  loglift
5421             ax  50760.000000  50760.000000  Default  30.0000  30.0000
1086              _   8392.000000   8392.000000  Default  29.0000  29.0000
26     organization  17488.000000  17488.000000  Default  28.0000  28.0000
18             line  19144.000000  19144.000000  Default  27.0000  27.0000
421           drive   2476.000000   2476.000000  Default  26.0000  26.0000
...             ...           ...           ...      ...      ...      ...
5208       familiar      0.010147      1.266583  Topic20 -11.2648   2.7421
11901             p      0.010147      1.266578  Topic20 -11.2648   2.7421
4497           loss      0.010147      1.266617  Topic20 -11.2648   2.7421
9087           phil      0.010147      1.266601  Topic20 -11.2648   2.7421
4711        capable      0.010147      1.266619  Topic20 -11.2648   2.7421

[857 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
1086      5  0.999819            _
406      19  0.905015       absurd
1588      3  0.998570       accept
1308     17  0.974783  acquisition
2073      3  0.017431       action
...     ...       ...          ...
45        1  0.372850         year
45        4  0.185013         year
45        6  0.430443         year
45        7  0.011669         year
2607     18  0.994252         zone

[1041 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 15, 10, 4, 1, 9, 8, 2, 14, 17, 6, 7, 3, 13, 19, 18, 20, 11, 12, 5])

### Each bubble on the left-hand side plot represents a topic. The larger the bubble, the more prevalent is that topic.
### A good topic model will have fairly big, non-overlapping bubbles scattered throughout the chart instead of being clustered in one quadrant.
### A model with too many topics, will typically have many overlaps, small sized bubbles clustered in one region of the chart.
### Move the cursor over one of the bubbles, the words and bars on the right-hand side will update. These words are the salient keywords that form the selected topic.
### We have successfully built a good looking topic model.